In [5]:
from typing_extensions import TypedDict, Optional
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from serpapi import GoogleSearch
from src.state.custom_state import FlightState
import datetime
import os 
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
# os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
# llm=ChatGroq(model="llama-3.3-70b-versatile")
# os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key= "*****************")

In [16]:
llm.invoke("Hello")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--7c40873d-9085-42b4-a3a5-7f6b3a2720d4-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [ ]:
def flight_tool(llm):

    # ---- Flight Tool ----
    
    class IATACode(BaseModel):
        iata_code: str = Field(description="The IATA airport code for the given city. For example, Jaipur -> 'JAI', Manchester -> 'MAN'.")

    class FlightReport(BaseModel):
        flight_information: Optional[str] = Field(
            description=  "Generate paragraph of flight information summarizing the options for the given trip" # "Summary of flight options for a user’s trip."
        )

    # -------- Flight Summary Prompt --------
    flight_prompt = ChatPromptTemplate.from_messages([
        ("system", """Please provide a summary of the following flight details. Include the following information:
        Here is the best flights information: {best_flights} 
        - Flight origin and destination
        - Departure and arrival times
        - Airline(s)
        - Flight duration
        - Price information (if available)
        - Any additional notable details e.g., stopovers, flight class, etc."""),
        ("human", "Best Flights Information:\n\n{best_flights}")
    ])

    structured_flight_llm = llm.with_structured_output(FlightReport)
    flight_report_generation = flight_prompt | structured_flight_llm

    def flight_finder(state: FlightState) -> dict:
        """Help to find best flight available from sources to destination"""
        start_city = state.get("start_city")
        destination_city = state.get("city")
        start_date = state.get("start_date")
        end_date = state.get("end_date")

        # Define system prompt for IATA
        system_msg = SystemMessage(content="Provide only the three-letter IATA code for the given city in a JSON object with the key '{iata_code}'. "
        "If the city has multiple airports, choose the primary or most widely used one based on passenger traffic or common usage. "
        "For example, if the city is Jaipur, respond with {'iata_code': 'JAI'}. If the city is London, respond with {'iata_code': 'LHR'}. "
        "If the city is Manchester, respond with {'iata_code': 'MAN'}."
        "if the city has no airport give code for nearest city.")

        # Get IATA for start city
        iata_start_output = llm.with_structured_output(IATACode).invoke([
            system_msg,
            HumanMessage(content=f"What is the IATA code for {start_city}?")
        ])
        departure_iata = iata_start_output.iata_code.strip().upper()

        # Get IATA for destination city
        iata_dest_output = llm.with_structured_output(IATACode).invoke([
            system_msg,
            HumanMessage(content=f"What is the IATA code for {destination_city}?")
        ])
        arrival_iata = iata_dest_output.iata_code.strip().upper()
        if not departure_iata or not arrival_iata:
            return {"flight_report": "Could not retrieve IATA codes for the cities."}
        # Flight search via SerpAPI
        search_params = {
            "api_key": "**************************",
            "engine": "google_flights",
            "hl": "en",
            "gl": "us",
            "departure_id": departure_iata,
            "arrival_id": arrival_iata,
            "outbound_date": start_date,
            "return_date": end_date,
            "currency": "USD",
        }

        # search = SerpAPIWrapper(serpapi_api_key=search_params["api_key"])
        # results = search.run(search_params)
        # print(results)
        search = GoogleSearch(search_params)
        results = search.get_dict()
        # Generate flight summary
        flight_info = flight_report_generation.invoke({"best_flights": results['best_flights']})
        return {"flight_informations": flight_info}
    
    return flight_finder


In [24]:
# Example state data
state = {
    "start_city": "New York",
    "city": "London",
    "start_date": "2025-07-15",
    "end_date": "2025-07-25"
}


In [25]:
finder = flight_tool(llm)

In [31]:
result = finder(state)

In [32]:
from pprint import pprint

In [34]:
pprint(result)

{'flight_informations': FlightReport(flight_information='Here are the flight options from John F. Kennedy International Airport (JFK) to Heathrow Airport (LHR) on July 15, 2025.  All flights are roundtrip and in economy class, with a price of $567. Options include British Airways (flight BA 178, 6h 55m flight duration), Virgin Atlantic (flight VS 26, 6h 50m), American Airlines (flight AA 142, 6h 55m), and Delta (flight DL 3, 7h 30m, overnight flight).  Legroom is 31 inches on all flights.  Wi-Fi is available for a fee on most flights except Delta, which offers free Wi-Fi.  In-seat power and USB outlets are available on most flights.')}
